In [ ]:
from mwasis import *  # noqa: F403
@tf.function(jit_compile=True)
def f(x):
    return x + 1


redirect = FDRedirector(STDERR)
redirect.start()
f(tf.constant(1))
redirect.stop();

# Wprowadzenie


## Problem decyzji i uczenia

Mamy do wyboru $N$ opcji. 
Każdy wybór daje jakąś nagrodę

Jak optymalnie wykorzystać wiedzę historyczna do podejmowania przyszłych decyzji?


# Przypomnienie

## Efekt danych

--- 

In [ ]:
import sympy
from sympy import symbols, Piecewise, integrate, simplify

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Math, Markdown



p = symbols('p')
papr = Piecewise((4*p, p < 1/2), (4 - 4*p, p >= 1/2))

pl_pr = sympy.plot(papr, (p, 0, 1),  xlabel='p', ylabel='f(p)', show=False)
#pl.show()

for no,nr in [(1,0),(3,1),(30,10)]:
    #Dla otrzymania 1 orła i 3 reszek otrzymamy funkcjię likelihood postaci
    display(Markdown(f'### Wypadło {no} orłow i {nr} reszek'))

    likelihood = sympy.binomial(no+nr,nr)*(1 - p)**nr * p**no
    pl_lik = sympy.plot(likelihood, (p, 0, 1),  xlabel='p', ylabel='f(data|p)', show=False);
    #pl.show()


    unnormalized_posterior = likelihood * papr
    marginal_likelihood = integrate(unnormalized_posterior, (p, 0, 1))
    post = unnormalized_posterior / marginal_likelihood

    t = sympy.latex(sympy.simplify(post))
    pl_pos=sympy.plot(post, (p, 0, 1),  xlabel='p', ylabel='f(p|data)', show=False) 
    #pl.show()

    plotgrid = sympy.plotting.PlotGrid(2, 2, pl_pr,pl_pos,pl_lik, show=False,size=(7,4.0))
    plotgrid.show()
    display(Markdown('\n --- \n'))
    display(Latex(r'$$'+t+r'$$'))
    display(Markdown('\n --- \n'))

## Sprzężone a priori

\alert{$P(\theta | D)$ i $P(\theta )$ należą do tej samej rodziny}




## Problem formalnie

**Multi-armed Bandit Problem (MAB)** 

<!-- - agent applies a sequence of actions $x_1, x_2, x_3,\ldots \in \mathcal{X}$ to a system
- applying action $x_t$, the agent observes an outcome
$y_t$, which the system randomly generates according to a conditional probability measure $q_\theta(\cdot | x_t)$
- The agent enjoys a reward $r_t = r(y_t)$, where $r$ is a known function.  
- The agent is initially uncertain about the
value of $\theta$ and represents his uncertainty using a prior distribution $p$.
 -->

- Agent stosuje sekwencję działań $x_1, x_2, x_3,\ldots \in \mathcal{X}$ na systemie.
- Stosując działanie $x_t$, agent obserwuje wynik $y_t$, który system generuje losowo zgodnie z warunkową miarą prawdopodobieństwa $q_\theta(\cdot | x_t)$.
- Agent otrzymuje nagrodę $r_t = r(y_t)$, gdzie $r$ jest znaną funkcją.
- Agent początkowo nie jest pewien wartości $\theta$ i reprezentuje swoją niepewność, używając rozkładu początkowego $p$.

## Nagroda

$$
E_{q_{\hat{\theta}}}[r(y_t) | x_t = x] = \sum_o q_{\hat{\theta}}(o|x) r(o).
$$


## Strategia zachłanna

Wybieramy opcje z najwieksza oczekiwana nagroda

$$a = \arg\max_x E R_x$$

### Problem

Brak eksploracji

## Strategia $\epsilon$-zachłanna

- Działamy zachlannie
- z pr $\epsilon$ wybieramy losowo

### Problem

Eksplorujemy, ale po długiej nauce dalej podejmujemy losowe decyzje

## Bayesowskie podejmowanie decyzji 

Dzisiejsze a posteriori to jutrzejsze a priori

### Sekwencja obserwacji

$$x_1,x_2,x_3,\ldots$$

### Aktualizacja wiedzy

Zakładamy klasę rozkładu $P$ z parametrami $\theta$ i a priori $p_0(\theta)$

- Każda obserwacja uaktualnia nam wiedzę 
- $p_1(\theta)\sim p(x_1|\theta)p_0(\theta)$
- $p(\theta_i)= p(\theta_{i-1}|x_i)$


## Thompson sampling

\begin{algorithm}[H]
\begin{scriptsize}
\caption{{\small $\text{BernTS}(K, \alpha, \beta)$}}\label{alg:BernoulliTS}
\begin{algorithmic}[1]
\For{$t=1,2,\ldots $}
\State \textcolor{blue}{\#sample model:}
\For{$k=1, \ldots, K$}
\State Sample $\hat{\theta}_k \sim \text{beta}(\alpha_k, \beta_k)$
\EndFor \\
\State \textcolor{blue}{\#select and apply action:}
\State $x_t \leftarrow \arg\max_k \hat{\theta}_k$
\State Apply $x_t$ and observe $r_t$ \\
\State \textcolor{blue}{\#update distribution:}
\State $(\alpha_{x_t}, \beta_{x_t}) \leftarrow  (\alpha_{x_t} + r_t, \beta_{x_t} + 1-r_t)$
\EndFor
\end{algorithmic}
\end{scriptsize}
\end{algorithm}

\url{https://doi.org/10.48550/arXiv.1707.02038}

## Strategia zachłanna

\begin{algorithm}[H]
\begin{scriptsize}
\caption{{\small  $\text{BernGreedy}(K, \alpha, \beta)$}}\label{alg:BernoulliGreedy}
\begin{algorithmic}[1]
\For{$t=1,2,\ldots $}
\State \textcolor{blue}{\#estimate model:}
\For{$k=1, \ldots, K$}
\State $\hat{\theta}_k \leftarrow \alpha_k / (\alpha_k + \beta_k)$
\EndFor \\
\State \textcolor{blue}{\#select and apply action:}
\State $x_t \leftarrow \arg\max_k \hat{\theta}_k$
\State Apply $x_t$ and observe $r_t$ \\
\State \textcolor{blue}{\#update distribution:}
\State $(\alpha_{x_t}, \beta_{x_t}) \leftarrow  (\alpha_{x_t} + r_t, \beta_{x_t} + 1-r_t)$
\EndFor
\end{algorithmic}
\end{scriptsize}
\end{algorithm}


# Wyniki numeryczne

### Przykład{.example}
Mamy trzy serwery, każdy obsługuje z rozkładem wykładniczym o nieznanej intensywności, chcemy wybrać najlepszy.

## Zastosowanie

* **Optymalizacja Wi-Fi:** Inteligentny wybór kanału i konfiguracji dla najlepszej wydajności sieci.
* **Wybór konfiguracji systemu:** Skuteczne testowanie i wybieranie optymalnych ustawień w złożonych systemach.
* **Uczenie ze wzmocnieniem (Reinforcement Learning):** Adaptacyjne algorytmy, które uczą się i dostosowują do dynamicznych środowisk.

## Podsumowanie

- Proponujemy rozkład nagrody per ramie
- Stosujemy sprzęzony rozkłd a priori
- Aktualizujemy wiedze o parametrach
- **Decyzja na podstawie próbki z a posteriori**

# {.standout}

Pytania